In [ ]:
# !pip install opencv-python
# !pip install seaborn

In [ ]:
# !pip install timm

In [ ]:
# !pip install albumentations

In [ ]:
# !pip install colorama

In [ ]:
# !pip install torchsummarypip install torch-summary

In [ ]:
# !pip install torchinfo

In [ ]:
# !pip install pynvml

In [ ]:
from pynvml import *
nvmlInit()
h = nvmlDeviceGetHandleByIndex(1)
info = nvmlDeviceGetMemoryInfo(h)
print(f'total    : {info.total}')
print(f'free     : {info.free}')
print(f'used     : {info.used}')

In [ ]:
#  article dependencies
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as Datasets
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
# import cv2
from tqdm.notebook import tqdm
from tqdm import tqdm as tqdm_regular
import seaborn as sns
from torchvision.utils import make_grid
import random
from torchinfo import summary
 # Input shape (3 channels, 32x32 pixels)


In [ ]:
#  configuring device
if torch.cuda.is_available():
  device = torch.device('cuda:0')
  print('Running on the GPU')
else:
  device = torch.device('cpu')
  print('Running on the CPU')

In [ ]:
import os
import gc
import cv2
import math
import copy
import time
import random
import glob
from matplotlib import pyplot as plt

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
import torchvision

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

# For Image Models
import timm

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np

train = []
size = (1536, 1536)

for filename in os.listdir("/home/21je0963/Competiton/data/train_img"):
    train.append(filename)

val_data = train[:100]
train_data = train[101:]

class CustomDataset(Dataset):
    def __init__(self, list_IDs, data_dir, transform=None):
        self.list_IDs = list_IDs
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.list_IDs)

    def __getitem__(self, index):
        ID = self.list_IDs[index]
        img_path = os.path.join(self.data_dir, ID)

        with Image.open(img_path) as img:
            img = img.convert('RGB')

        if self.transform:
            img = self.transform(img)

        return img

data_dir = '/home/21je0963/Competiton/data/train_img/'

transform = transforms.Compose([
    transforms.Resize((768, 768)),
    transforms.ToTensor(),
])

train_custom_dataset = CustomDataset(train_data, data_dir, transform=transform)
val_custom_dataset = CustomDataset(val_data, data_dir, transform=transform)


In [ ]:
# class UBCDataset(Dataset):
#     def __init__(self, df, transforms=None):
#         self.df = df
#         self.file_names = df['file_path'].values
#         self.labels = df['label'].values
#         self.transforms = transforms
        
#     def __len__(self):
#         return len(self.df)
    
#     def __getitem__(self, index):
#         img_path = self.file_names[index]
#         img = cv2.imread(img_path)
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#         label = self.labels[index]
        
#         if self.transforms:
#             img = self.transforms(image=img)["image"]
            
#         return {
#             'image': img,
#             'label': torch.tensor(label, dtype=torch.long)
#         }

In [ ]:
# #  defining encoder
# class Encoder(nn.Module):
#   def __init__(self, in_channels=3, out_channels=16, latent_dim=200, act_fn=nn.ReLU()):
#     super().__init__()

#     self.net = nn.Sequential(
#         nn.Conv2d(in_channels, out_channels, 3, padding=1), # (32, 32)
#         act_fn,
#         nn.Conv2d(out_channels, out_channels, 3, padding=1), 
#         act_fn,
#         nn.Conv2d(out_channels, 2*out_channels, 3, padding=1, stride=2), # (16, 16)
#         act_fn,
#         nn.Conv2d(2*out_channels, 2*out_channels, 3, padding=1),
#         act_fn,
#         nn.Conv2d(2*out_channels, 4*out_channels, 3, padding=1, stride=2), # (8, 8)
#         act_fn,
#         nn.Conv2d(4*out_channels, 4*out_channels, 3, padding=1),
#         act_fn,
#         nn.Flatten(),
#         nn.Linear(4*out_channels*8*8, latent_dim),
#         act_fn
#     )

#   def forward(self, x):
#     x = x.view(-1, 3, 32, 32)
#     output = self.net(x)
#     return output


# #  defining decoder
# class Decoder(nn.Module):
#   def __init__(self, in_channels=3, out_channels=16, latent_dim=200, act_fn=nn.ReLU()):
#     super().__init__()

#     self.out_channels = out_channels

#     self.linear = nn.Sequential(
#         nn.Linear(latent_dim, 4*out_channels*8*8),
#         act_fn
#     )

#     self.conv = nn.Sequential(
#         nn.ConvTranspose2d(4*out_channels, 4*out_channels, 3, padding=1), # (8, 8)
#         act_fn,
#         nn.ConvTranspose2d(4*out_channels, 2*out_channels, 3, padding=1, 
#                            stride=2, output_padding=1), # (16, 16)
#         act_fn,
#         nn.ConvTranspose2d(2*out_channels, 2*out_channels, 3, padding=1),
#         act_fn,
#         nn.ConvTranspose2d(2*out_channels, out_channels, 3, padding=1, 
#                            stride=2, output_padding=1), # (32, 32)
#         act_fn,
#         nn.ConvTranspose2d(out_channels, out_channels, 3, padding=1),
#         act_fn,
#         nn.ConvTranspose2d(out_channels, in_channels, 3, padding=1)
#     )

#   def forward(self, x):
#     output = self.linear(x)
#     output = output.view(-1, 4*self.out_channels, 8, 8)
#     output = self.conv(output)
#     return output


# #  defining autoencoder
# class Autoencoder(nn.Module):
#   def __init__(self, encoder, decoder):
#     super().__init__()
#     self.encoder = encoder
#     self.encoder.to(device)

#     self.decoder = decoder
#     self.decoder.to(device)

#   def forward(self, x):
#     encoded = self.encoder(x)
#     decoded = self.decoder(encoded)
#     return decoded

In [ ]:
device

In [ ]:

# encoder = Encoder()
# encoder.to(device)
# summary(encoder, (3, 32, 32), device = device) 
class Encoder(nn.Module):
    def __init__(self, in_channels=3, out_channels=16, latent_dim=200, act_fn=nn.ReLU()):
        super().__init__()

        self.net = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),  # (768, 768)
            act_fn,
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            act_fn,
            nn.Conv2d(out_channels, 2*out_channels, 3, padding=1, stride=2),  # (384, 384)
            act_fn,
            nn.Conv2d(2*out_channels, 2*out_channels, 3, padding=1),
            act_fn,
            nn.Conv2d(2*out_channels, 4*out_channels, 3, padding=1, stride=2),  # (192, 192)
            act_fn,
            nn.Conv2d(4*out_channels, 4*out_channels, 3, padding=1),
            act_fn,
            nn.Flatten(),
            nn.Linear(4*out_channels*192*192, latent_dim),
            act_fn
        )

    def forward(self, x):
        x = x.view(-1, 3, 768, 768)
        output = self.net(x)
        return output

class Decoder(nn.Module):
    def __init__(self, in_channels=3, out_channels=16, latent_dim=200, act_fn=nn.ReLU()):
        super().__init__()

        self.out_channels = out_channels

        self.linear = nn.Sequential(
            nn.Linear(latent_dim, 4 * out_channels * 192 * 192),
            act_fn
        )

        self.conv = nn.Sequential(
            nn.ConvTranspose2d(4 * out_channels, 4 * out_channels, 3, padding=1),  # (192, 192)
            act_fn,
            nn.ConvTranspose2d(4 * out_channels, 2 * out_channels, 3, padding=1, stride=2, output_padding=1),  # (384, 384)
            act_fn,
            nn.ConvTranspose2d(2 * out_channels, 2 * out_channels, 3, padding=1),
            act_fn,
            nn.ConvTranspose2d(2 * out_channels, out_channels, 3, padding=1, stride=2, output_padding=1),  # (768, 768)
            act_fn,
            nn.ConvTranspose2d(out_channels, out_channels, 3, padding=1),
            act_fn,
            nn.ConvTranspose2d(out_channels, in_channels, 3, padding=1)
        )

    def forward(self, x):
        output = self.linear(x)
        output = output.view(-1, 4 * self.out_channels, 192, 192)
        output = self.conv(output)
        return output



class Autoencoder(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
encoder = Encoder()
encoder.to(device)
summary(encoder, (3, 768, 768), device = device) 

In [ ]:
decoder = Decoder()
decoder.to(device)
summary(decoder, (1,200), device = device)

In [ ]:
autoencoder = Autoencoder(encoder, decoder)
autoencoder.to(device)
summary(autoencoder, (3,1536, 1536), device = device)

In [ ]:
from pynvml import *
nvmlInit()
h = nvmlDeviceGetHandleByIndex(1)
info = nvmlDeviceGetMemoryInfo(h)
print(f'total    : {info.total}')
print(f'free     : {info.free}')
print(f'used     : {info.used}')

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define a custom Laplacian filter bank layer.
class LaplacianFilterBank(nn.Module):
    def __init__(self, in_channels, num_filters, filter_sizes):
        super(LaplacianFilterBank, self).__init__()

        self.filters = nn.ModuleList()
        for filter_size in filter_sizes:
            self.filters.append(nn.Conv2d(in_channels, num_filters, filter_size, padding=(filter_size - 1) // 2))

    def forward(self, x):
        subbands = [filter(x) for filter in self.filters]
        return subbands

# Custom spatial frequency loss module.
class SpatialFrequencyLoss(nn.Module):
    def __init__(self, in_channels, num_filters, filter_sizes):
        super(SpatialFrequencyLoss, self).__init__()
        self.filter_bank = LaplacianFilterBank(in_channels, num_filters, filter_sizes).to(device)

    def forward(self, input_image, reconstructed_image):
        input_subbands = self.filter_bank(input_image)
        reconstructed_subbands = self.filter_bank(reconstructed_image)

        # Compute the MSE loss for each subband and sum them up.
        spatial_losses = [F.mse_loss(reconstructed_subband, input_subband) for reconstructed_subband, input_subband in zip(reconstructed_subbands, input_subbands)]
        spatial_frequency_loss = torch.sum(torch.stack(spatial_losses))

        return spatial_frequency_loss

# Example usage:


# input_image = torch.randn(1, 3, 768, 768)  # Replace with your actual input image
# reconstructed_image = torch.randn(1, 3, 768, 768)  # Replace with your actual reconstructed image

# loss = spatial_loss(input_image, reconstructed_image)


In [ ]:
class ConvolutionalAutoencoder():
  def __init__(self, autoencoder):
    self.network = autoencoder
    self.optimizer = torch.optim.Adam(self.network.parameters(), lr=1e-3)
    self.file1 = open("Train_logs_Spatial.txt",'w')
    self.file1.write("Training_started..............")
    self.file1.close()
      
    
  def train(self, loss_function, epochs, batch_size, training_set, validation_set, test_dataset):
    #  creating log
    log_dict = {
        'training_loss_per_batch': [],
        'validation_loss_per_batch': [],
        'visualizations': []
    } 

    #  defining weight initialization function
    def init_weights(module):
      if isinstance(module, nn.Conv2d):
        torch.nn.init.xavier_uniform_(module.weight)
        module.bias.data.fill_(0.01)
      elif isinstance(module, nn.Linear):
        torch.nn.init.xavier_uniform_(module.weight)
        module.bias.data.fill_(0.01)

    #  initializing network weights
    self.network.apply(init_weights)

    #  creating dataloaders
    
    train_loader = DataLoader(training_set, batch_size=8, shuffle=True)
    val_loader = DataLoader(validation_set, batch_size=8, shuffle=True)
    test_loader = DataLoader(test_dataset, 10)
    #  setting convnet to training mode
    self.network.train()
    self.network.to(device)

    for epoch in range(epochs):
      print(f'Epoch {epoch+1}/{epochs}')
      train_losses = []

      #------------
      #  TRAINING
      #------------
      print('training...')
      for images in tqdm(train_loader):
        #  zeroing gradients
        self.optimizer.zero_grad()
        #  sending images to device
        # print(len(images))
        images = images.to(device)
          
        #  reconstructing images
        output = self.network(images)
        #  computing loss
        loss = loss_function(output, images.view(-1, 3, 768, 768))
        #  calculating gradients
        loss.backward()
        #  optimizing weights
        self.optimizer.step()

        #--------------
        # LOGGING
        #--------------
        log_dict['training_loss_per_batch'].append(loss.item())

      #--------------
      # VALIDATION
      #--------------
      print('validating...')
      for val_images in tqdm(val_loader):
      
        with torch.no_grad():
          #  sending validation images to device
          val_images = val_images.to(device)
          #  reconstructing images
          output = self.network(val_images)
          #  computing validation loss
          val_loss = loss_function(output, val_images.view(-1, 3, 768, 768))

        #--------------
        # LOGGING
        #--------------
        log_dict['validation_loss_per_batch'].append(val_loss.item())
      
     

      #--------------
      # VISUALISATION
      #--------------
      print(f'training_loss: {round(loss.item(), 4)} validation_loss: {round(val_loss.item(), 4)}')
      l = f'training_loss: {round(loss.item(), 4)} validation_loss: {round(val_loss.item(), 4)}'
      self.file1 = open("Train_logs_Spatial.txt",'a')
      self.file1.write(l)
      self.file1.close()
      for test_images in test_loader:
        #  sending test images to device
        test_images = test_images.to(device)
        with torch.no_grad():
          #  reconstructing test images
          reconstructed_imgs = self.network(test_images)
        #  sending reconstructed and images to cpu to allow for visualization
        reconstructed_imgs = reconstructed_imgs.cpu()
        test_images = test_images.cpu()

        #  visualisation
        imgs = torch.stack([test_images.view(-1, 3, 768, 768), reconstructed_imgs], 
                          dim=1).flatten(0,1)
        grid = make_grid(imgs, nrow=10, normalize=True, padding=1)
        grid = grid.permute(1, 2, 0)
        plt.figure(dpi=170)
        plt.title('Original/Reconstructed')
        plt.imshow(grid)
        log_dict['visualizations'].append(grid)
        plt.axis('off')
        plt.show()
    
    # self.file1.close() 
    return log_dict

  def autoencode(self, x):
    return self.network(x)

  def encode(self, x):
    encoder = self.network.encoder
    return encoder(x)
  
  def decode(self, x):
    decoder = self.network.decoder
    return decoder(x)

In [ ]:
# model = ConvolutionalAutoencoder(Autoencoder(Encoder(), Decoder()))
# model.summary()
test_dataset = CustomDataset(val_data[:10], data_dir, transform=transform)

In [ ]:
in_channels = 3  # Input has 3 channels
num_filters = 16  # Number of filters in the filter bank
filter_sizes = [3, 5]  # Filter sizes to capture different spatial frequencies

loss_function = SpatialFrequencyLoss(in_channels, num_filters, filter_sizes)

In [ ]:
model = ConvolutionalAutoencoder(autoencoder)

log_dict = model.train(loss_function, epochs=100, batch_size=4, training_set=train_custom_dataset, validation_set=val_custom_dataset, test_dataset = test_dataset)

In [ ]:
torch.save(autoencoder, 'spatial_model_weights.pth')

In [ ]:
modell = torch.load('spatial_model_weights.pth')

# summary(model, (3, 768, 768))
test_loader = DataLoader(test_dataset, 10)

for test_images in test_loader:
        #  sending test images to device
        test_images = test_images.to(device)
        with torch.no_grad():
          #  reconstructing test images
          reconstructed_imgs = modell(test_images)
        #  sending reconstructed and images to cpu to allow for visualization
        reconstructed_imgs = reconstructed_imgs.cpu()
        test_images = test_images.cpu()

        #  visualisation
        imgs = torch.stack([test_images.view(-1, 3, 768, 768), reconstructed_imgs], 
                          dim=1).flatten(0,1)
        grid = make_grid(imgs, nrow=10, normalize=True, padding=1)
        grid = grid.permute(1, 2, 0)
        plt.figure(dpi=170)
        plt.title('Original/Reconstructed')
        plt.imshow(grid)
        log_dict['visualizations'].append(grid)
        plt.axis('off')
        plt.show()